In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path
from plotutil import show_im,overlap, show_im_rgb
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton
import math
import networkx as nx
plate=13
date1='0703_1157'
date2='0703_1557'
date3='0703_1957'
row=6
column=10

In [2]:
imtab1=np.load(f'Data/imbackrem_{date1}_{plate}_{row}_{column}.npy')
imtab2=np.load(f'Data/imbackrem_{date2}_{plate}_{row}_{column}.npy')
imtab3=np.load(f'Data/imbackrem_{date3}_{plate}_{row}_{column}.npy')
frangi1=np.load(f'Data/frangised_{date1}_{plate}_{row}_{column}.npy')
frangi2=np.load(f'Data/frangised_{date2}_{plate}_{row}_{column}.npy')
frangi3=np.load(f'Data/frangised_{date3}_{plate}_{row}_{column}.npy')
dilated1=np.load(f'Data/dilated_{date1}_{plate}_{row}_{column}.npy')
dilated2=np.load(f'Data/dilated_{date2}_{plate}_{row}_{column}.npy')
# dilated3=np.load(f'Data/dilated_{date3}_{plate}_{row}_{column}.npy')
skeleton1=np.load(f'Data/skeletonizedpruned_{date1}_{plate}_{row}_{column}.npy')
skeleton2=np.load(f'Data/skeletonizedpruned_{date2}_{plate}_{row}_{column}.npy')
skeleton3=np.load(f'Data/skeletonizedpruned_{date3}_{plate}_{row}_{column}.npy')
width2=np.load(f'Data/width_{date2}_{plate}_{row}_{column}.npy')

In [24]:
graph_tab1=generate_graph_tab_from_skeleton(skeleton1)
graph_tab2=generate_graph_tab_from_skeleton(skeleton2)
graph_tab3=generate_graph_tab_from_skeleton(skeleton3)
nx_graph_t0m1,pos_t0m1=generate_nx_graph_from_skeleton(skeleton1)
nx_graph_t0,pos_t0=generate_nx_graph_from_skeleton(skeleton2)
nx_graph_t0p1,pos_t0p1=generate_nx_graph_from_skeleton(skeleton3)

In [4]:
def node_dist(node1,node2,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,show=False):
    sparse_cross1=sparse.dok_matrix((3000,4096), dtype=bool)
    sparse_cross2=sparse.dok_matrix((3000,4096), dtype=bool)
    for edge in nx_graph_tm1.edges(node1):
        list_pixel=nx_graph_tm1.get_edge_data(*edge)['pixel_list']
        if (pos_tm1[node1]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
        for pixel in list_pixel[:20]:
            sparse_cross1[pixel]=1
    for edge in nx_graph_t.edges(node2):
        list_pixel=nx_graph_t.get_edge_data(*edge)['pixel_list']
        if (pos_t[node2]!=list_pixel[0]).any():
            list_pixel=list(reversed(list_pixel))
        for pixel in list_pixel[:20]:
            sparse_cross2[pixel]=1
    kernel = np.ones((3,3),np.uint8)
    dilation1 = cv2.dilate(sparse_cross1.todense().astype(np.uint8),kernel,iterations = 3)
    dilation2 = cv2.dilate(sparse_cross2.todense().astype(np.uint8),kernel,iterations = 3)
    if show:
        plt.imshow(dilation1)
        plt.imshow(dilation2,alpha=0.5)
        plt.show()
    return(np.linalg.norm(dilation1-dilation2))

In [5]:
def first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t):
    corresp={}
    ambiguous=set()
    to_remove=set()
    degree_3sup_nodes_tm1 = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)>=3]
    degree_3sup_nodes_t = [node for node in nx_graph_t.nodes if nx_graph_t.degree(node)>=3]
    for node1 in degree_3sup_nodes_tm1:
        mini=np.inf
        for node2 in degree_3sup_nodes_t:
            distance=np.linalg.norm(pos_tm1[node1]-pos_t[node2])
            if distance<mini:
                mini=distance
                identifier=node2
        if mini<30:
            if identifier in corresp.values():
                ambiguous.add(node1)
#                     print(node1,'node_dientified_two_times')
            corresp[node1]=identifier
        else:
            to_remove.add(node1)
#                 print(node1,mini,'node_none_iden')
    while len(ambiguous)>0:
        node=ambiguous.pop()
        identifier=corresp[node]
        candidates = [nod for nod in corresp.keys() if corresp[nod]==identifier]
        mini=np.inf
        for candidate in candidates:
            distance=node_dist(candidate,identifier,nx_graph_tm1,nx_graph_t,pos_tm1,pos_t)
            if distance < mini:
                right_candidate=candidate
                mini=distance
#         print(mini,right_candidate)
        for candidate in candidates:
            if candidate!= right_candidate:
                corresp.pop(candidate)
                to_remove.add(candidate)
                ambiguous.discard(candidate)
    return(corresp,to_remove)
    

In [6]:
def relabel_nodes(corresp,nx_graph_t,pos_t):
    invert_corresp={}
    new_pos = {}
    for key in corresp.keys():
        invert_corresp[corresp[key]]=key
    maxi=max(corresp.keys())+1
    def mapping(node):
        if node in invert_corresp.keys():
            return(invert_corresp[node])
        else:
            return (maxi+node)
    for node in nx_graph_t.nodes:
        pos=pos_t[node]
        if node in invert_corresp.keys():
            new_pos[invert_corresp[node]]=pos
        else:
            new_pos[maxi+node]=pos
    new_graph=nx.relabel_nodes(nx_graph_t,mapping)
    return(new_pos,new_graph)

In [7]:
def reconnect_degree_2(nx_graph,pos):
    degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]
    while len(degree_2_nodes)>0:
        for node in degree_2_nodes:
            neighbours = list(nx_graph.neighbors(node))
            right_n = neighbours[0]
            left_n = neighbours[1]
            right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
            left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
            if np.any(right_edge[0]!=pos[node]):
                right_edge = list(reversed(right_edge))
            if np.any(left_edge[-1]!=pos[node]):
                left_edge = list(reversed(left_edge))
            pixel_list = left_edge+right_edge[1:]
            info={'weight':len(pixel_list),'pixel_list':pixel_list}
            if right_n!=left_n:
                connection_data=nx_graph.get_edge_data(right_n,left_n)
                if connection_data is None or connection_data['weight']>info['weight']:
                    if not connection_data is None:
                        nx_graph.remove_edge(right_n,left_n,info)
                    nx_graph.add_edges_from([(right_n,left_n,info)])
            nx_graph.remove_node(node)
            degree_2_nodes = [node for node in nx_graph.nodes if nx_graph.degree(node)==2]

In [8]:
from copy import deepcopy
def clean_nodes(nx_graph1,to_remove,pos):
    nx_graph=deepcopy(nx_graph1)
    is_hair = True
    degree2_to_reconnect=[]
    while is_hair:
        is_hair=False
        to_remove_possibly=list(to_remove)
        for node in to_remove_possibly:
            neighbours = nx_graph.neighbors(node)
            candidate_to_remove=[]
            weight_candidate=[]
            for neighbour in neighbours:
                if nx_graph.degree(neighbour)==1:
                    is_hair=True
                    candidate_to_remove.append(neighbour)
                    weight_candidate.append(len(nx_graph.get_edge_data(node,neighbour)['pixel_list']))
            if len(candidate_to_remove)>0:
                node_to_remove=candidate_to_remove[np.argmin(weight_candidate)]
                nx_graph.remove_node(node_to_remove)
#                 print(node_to_remove,' is removed')
                if nx_graph.degree(node)==2:
                    to_remove.discard(node)
                    degree2_to_reconnect.append(node)
    reconnect_degree_2(nx_graph,pos)
#     for node in degree2_to_reconnect:
#         neighbours = list(nx_graph.neighbors(node))
#         right_n = neighbours[0]
#         left_n = neighbours[1]
#         right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
#         left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
#         if np.any(right_edge[0]!=pos[node]):
#             right_edge = list(reversed(right_edge))
#         if np.any(left_edge[-1]!=pos[node]):
#             left_edge = list(reversed(left_edge))
#         pixel_list = left_edge+right_edge[1:]
#         info={'weight':len(pixel_list),'pixel_list':pixel_list}
#         if right_n!=left_n:
#             connection_data=nx_graph.get_edge_data(right_n,left_n)
#             if connection_data is None or connection_data['weight']>info['weight']:
#                 if not connection_data is None:
#                     nx_graph.remove_edge(right_n,left_n,info)
#                     print(right_n,left_n,'is removed')
#                 nx_graph.add_edges_from([(right_n,left_n,info)])
#                 print(right_n,left_n,'is added')
#         print(node,' is removed')
#         nx_graph.remove_node(node)
#     print(list(nx_graph.neighbors(5)))
    reconnect_degree_2(nx_graph,pos)
    for node in to_remove:
        if node in nx_graph:
            neighbours = list(nx_graph.neighbors(node))
            candidate_to_fuse=[]
            weight_candidate=[]
            for neighbour in neighbours:
                candidate_to_fuse.append(neighbour)
                weight_candidate.append(len(nx_graph.get_edge_data(node,neighbour)['pixel_list']))
            node_to_fuse=candidate_to_fuse[np.argmin(weight_candidate)]
            for neighbour in neighbours:
                right_n = node_to_fuse
                left_n = neighbour
    #             print(right_n,left_n)
    #             print(right_n!=left_n)
                right_edge = nx_graph.get_edge_data(node,right_n)['pixel_list']
                left_edge = nx_graph.get_edge_data(node,left_n)['pixel_list']
                if np.any(right_edge[0]!=pos[node]):
                    right_edge = list(reversed(right_edge))
                if np.any(left_edge[-1]!=pos[node]):
                    left_edge = list(reversed(left_edge))
                pixel_list = left_edge+right_edge[1:]
                info={'weight':len(pixel_list),'pixel_list':pixel_list}
                if right_n!=left_n:
                    connection_data=nx_graph.get_edge_data(right_n,left_n,info)
                    if connection_data is None or connection_data['weight']>info['weight']:
                        if not connection_data is None:
                            nx_graph.remove_edge(right_n,left_n,info)

                        nx_graph.add_edges_from([(right_n,left_n,info)])
            nx_graph.remove_node(node)
    return(nx_graph)
                    

In [9]:
# corresp,to_remove=first_identification(nx_graph_t0m1,nx_graph_t0,pos_t0m1,pos_t0)

In [10]:
# nx_graph_new=clean_nodes(nx_graph_t0m1,to_remove,pos_t0m1)

In [11]:
# pos_t0=relabel_nodes(corresp,nx_graph_t0,pos_t0)

In [12]:
def orient(pixel_list,root_pos):
    if np.all(root_pos==pixel_list[0]):
        return(pixel_list)
    else:
        return list(reversed(pixel_list))

In [20]:
def second_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    corresp,to_remove=first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t)
    nx_graph_tm1=clean_nodes(nx_graph_tm1,to_remove,pos_tm1)
#     pos_t,nx_graph_t=relabel_nodes(corresp,nx_graph_t,pos_t)
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for tip in tips:
#         print('tip',pos_tm1[tip],tip)
        root=next(nx_graph_tm1.neighbors(tip))
        branch_tm1=orient(nx_graph_tm1.get_edge_data(root,tip)['pixel_list'],pos_tm1[root])
        mini=np.inf
        for neighbours_t in nx_graph_t.neighbors(root):
            branch_candidate=orient(nx_graph_t.get_edge_data(root,neighbours_t)['pixel_list'],pos_t[root])
            length_for_id = min(length_id,len(branch_tm1),len(branch_candidate))
            distance=(np.linalg.norm(np.array(branch_tm1[:length_for_id])-np.array(branch_candidate[:length_for_id])))/length_for_id
#             print(pos_t[root],pos_t[neighbours_t],distance)
            if distance<mini:
                next_node=neighbours_t
                mini=distance
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        while nx_graph_t.degree(current_node)!=1: #Careful : if there is a cycle with low angle this might loop indefinitely but unprobable
            mini=np.inf
            origin_vector = last_branch[0]-last_branch[min(length_id,len(last_branch)-1)]
            candidate_vectors=[]
            for neighbours_t in nx_graph_t.neighbors(current_node):
                if neighbours_t!=last_node:
                    branch_candidate=np.array(orient(nx_graph_t.get_edge_data(current_node,neighbours_t)['pixel_list'],pos_t[current_node]))
#                     print(last_branch[0],branch_candidate[0])
                    candidate_vector = branch_candidate[min(length_id,len(branch_candidate)-1)]-branch_candidate[0]
                    unit_vector_origin = origin_vector / np.linalg.norm(origin_vector)
                    unit_vector_candidate = candidate_vector / np.linalg.norm(candidate_vector)
                    candidate_vectors.append(unit_vector_candidate)
                    dot_product = np.dot(unit_vector_origin, unit_vector_candidate)
                    angle = np.arccos(dot_product)
                    if angle<mini:
                        mini=angle
                        next_node=neighbours_t
#                     print('angle',dot_product,pos_t[last_node],pos_t[current_node],pos_t[neighbours_t],angle/(2*np.pi)*360)
            competitor = np.arccos(np.dot(candidate_vectors[0],-candidate_vectors[1]))
#             print('competitor',competitor/(2*np.pi)*360)
            if mini<competitor:
                current_node,last_node=next_node,current_node
            else:
                corresp[tip]=current_node
                break
        corresp[tip]=current_node
    pos_t,nx_graph_t=relabel_nodes(corresp,nx_graph_t,pos_t)
    return(pos_t,nx_graph_t,nx_graph_tm1,corresp)
        

In [30]:
post_t0_clean,nx_graph_t0_clean,nx_graph_t0m1_clean=second_identification(nx_graph_t0m1,nx_graph_t0,pos_t0m1,pos_t0,50)
skeleton_clean_t0m1=generate_skeleton(nx_graph_t0m1_clean).todense()
# pos_t0p1,nx_graph_t0p1,nx_graph_t0=second_identification(nx_graph_t0,nx_graph_t0p1,pos_t0,pos_t0p1,50)
skeleton_clean_t0=generate_skeleton(nx_graph_t0).todense()

In [28]:
def plot_nodes(nx_graph,pos,skeleton):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(skeleton)
    bbox_props = dict(boxstyle="circle", fc="white")
    for node in nx_graph.nodes:
        t = ax.text(pos[node][1], pos[node][0], str(node), ha="center", va="center",
                    size=5,
                    bbox=bbox_props)
    plt.show()

In [31]:
plot_nodes(nx_graph_t0,pos_t0,skeleton_clean_t0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plot_nodes(nx_graph_t0m1,pos_t0m1,skeleton_clean_t0m1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
from plotutil import overlap
overlap(skeleton_clean_t0m1.todense(),skeleton2)

In [ ]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(skeleton2)
bbox_props = dict(boxstyle="circle", fc="white")
for node in nx_graph_t0.nodes:
    t = ax.text(pos_t0[node][1], pos_t0[node][0], str(node), ha="center", va="center",
                size=5,
                bbox=bbox_props)
plt.show()

In [ ]:
for node1 in nx_graph_new.nodes:
    if nx_graph_new.degree[node1]==2:
        print(node1)

In [ ]:
list(nx_graph_new.neighbors(5))

In [ ]:
pos_t0m1[7]

In [ ]:
pos_t0m1[3]

In [ ]:
corresp[9]

In [ ]:
skeleton_clean_t0m1=generate_skeleton(nx_graph_new)

In [ ]:
problem=(1729,646)

In [ ]:
problem in [tuple(value) for value in pos_t0m1.values()]

In [ ]:
list(nx_graph_t0.neighbors(48))
nx_graph_t0.get_edge_data(46,48)['pixel_list']

In [ ]:
corresp[65]

In [ ]:
pos_t0[65]

In [ ]:
%matplotlib widget
node_dist(64,85,nx_graph_t0,nx_graph_t0p1,pos_t0,pos_t0p1,show=True)

In [ ]:
corresp[6]

In [ ]:
%matplotlib widget

node_dist(15,27,nx_graph1,nx_graph2,pos1,pos2)

In [ ]:
pos1[15]

In [ ]:
pos1[6]